In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# import the necessary package

In [3]:
movies = pd.read_csv("PMA_blockbuster_movies.csv")
movies.head()
# read the data into python by pandas
# show some details of the datases

,poster_url,rt_audience_score,rt_freshness,2015_inflation,adjusted,genres,Genre_1,Genre_2,Genre_3,imdb_rating,length,rating,release_date,studio,title,worldwide_gross,year
0,http://resizing.flixster.com/gxRJwetP1eNIrPR6x...,4.3,89.0,-0.26%,"$712,903,691.09",Sci-Fi\nAdventure\nAction,Sci-Fi,Adventure,Action,7.8,136.0,PG-13,04-Apr-14,Marvel Studios,Captain America: The Winter Soldier,"$714,766,572.00",2014.0
1,http://resizing.flixster.com/gDtbA1iPxTYEjBZeS...,4.2,90.0,-0.26%,"$706,988,165.89",Sci-Fi\nDrama\nAction,Sci-Fi,Drama,Action,7.7,130.0,PG-13,11-Jul-14,20th Century Fox,Dawn of the Planet of the Apes,"$708,835,589.00",2014.0
2,http://resizing.flixster.com/YrF_OeTQx3bXNsMLI...,4.4,91.0,-0.26%,"$772,158,880.00",Sci-Fi\nAdventure\nAction,Sci-Fi,Adventure,Action,8.1,121.0,PG-13,01-Aug-14,Marvel Studios,Guardians of the Galaxy,"$774,176,600.00",2014.0
3,http://resizing.flixster.com/l9yjA-72sZMYECeOj...,4.2,72.0,-0.26%,"$671,220,455.10",Sci-Fi\nAdventure,Sci-Fi,Adventure,NaN,8.7,169.0,PG-13,07-Nov-14,Paramount Pictures / Warner Bros.,Interstellar,"$672,974,414.00",2014.0
4,http://resizing.flixster.com/YukULOFULUesVZccN...,3.8,49.0,-0.26%,"$756,677,675.77",Family\nAdventure\nAction,Family,Adventure,Action,7.1,97.0,PG,30-May-14,Walt Disney Pictures,Maleficent,"$758,654,942.00",2014.0


In [4]:
movies = movies.drop(["poster_url","2015_inflation","release_date","title","worldwide_gross","year","studio"],axis =1)
# drop some useless features. 
# since the box office has been adjusted, 2015_inflation and worldwide_gross is not necessary in the prediction. the data in 2015_inflation mutiply the data in word_gross is the data in adjusted field.
# since good film will not be effected by time, yaer is deleted
# poster_url, title and release_date are not related so drop them.
# studio may affect the box office like marvel and universal, but we do not consider it in this prediction.

In [6]:
movies = movies.dropna(subset=["adjusted"])
# delete the missing value in Dependent variable column.

In [7]:
movies = movies.drop(["genres","Genre_2","Genre_3"],axis=1)
# choose the main type of movie as a variable

In [9]:
np.random.seed(0) 
missing_values_count = movies.isnull().sum()
print(missing_values_count)
# through counting missing value, it is interseting to note that there is no missing value
# then the next step is using get_dummies idea to clean the dataset. 

rt_audience_score    0
rt_freshness         0
adjusted             0
Genre_1              0
imdb_rating          0
length               0
rating               0
dtype: int64


In [10]:
movies_genre = pd.get_dummies(movies["Genre_1"],prefix='genre')
# use get_dummies to process the genre

In [11]:
movies_rating = pd.get_dummies(movies["rating"],prefix='rating')
# use get_dummies to process the rating

In [12]:
movies = pd.concat([movies.drop(["Genre_1","rating"], axis=1),movies_genre,movies_rating], axis=1)
# merge all the dataframe

In [13]:
movies['adjusted'] = movies['adjusted'].str.replace(',', '')
movies['adjusted'] = movies['adjusted'].str.replace('$', '')
movies['adjusted'] = movies['adjusted'].astype(float)
# convert box office's type into float

In [14]:
movies.head()
# all cleaning steps and feature engineering steps are done, the next step will be separating the target variable.

,rt_audience_score,rt_freshness,adjusted,imdb_rating,length,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,...,genre_Romance,genre_Sci-Fi,genre_Sport,genre_Thriller,genre_War,genre_Western,rating_G,rating_PG,rating_PG-13,rating_R
0,4.3,89.0,7.129037e+08,7.8,136.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,4.2,90.0,7.069882e+08,7.7,130.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,4.4,91.0,7.721589e+08,8.1,121.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,4.2,72.0,6.712205e+08,8.7,169.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,3.8,49.0,7.566777e+08,7.1,97.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
movies.describe()

,rt_audience_score,rt_freshness,adjusted,imdb_rating,length,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,...,genre_Romance,genre_Sci-Fi,genre_Sport,genre_Thriller,genre_War,genre_Western,rating_G,rating_PG,rating_PG-13,rating_R
count,398.000000,398.000000,3.980000e+02,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,...,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,3.478894,69.381910,5.428198e+08,7.053266,119.253769,0.007538,0.032663,0.015075,0.135678,0.042714,...,0.118090,0.123116,0.017588,0.138191,0.025126,0.007538,0.065327,0.319095,0.369347,0.246231
std,0.391222,23.186895,3.407107e+08,0.834777,22.973721,0.086601,0.177978,0.122006,0.342878,0.202465,...,0.323121,0.328983,0.131614,0.345535,0.156704,0.086601,0.247412,0.466713,0.483235,0.431357
min,1.700000,0.000000,1.100475e+08,4.400000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,54.000000,2.936418e+08,6.500000,102.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.500000,75.500000,4.880479e+08,7.000000,117.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.700000,88.000000,6.985807e+08,7.675000,131.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,4.500000,100.000000,3.025615e+09,9.000000,201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
# It can be seen from the dataset that all the box office are higher than $100000000, which means nearly all the sample is successful in box office
# I decide to see it as a classification issue, and use oversampling to sampling.
# SVR,XGBoosting and GBDT are used in the prediction. Finally,voting classifier will be used in the end.

In [17]:
movies_adjusted = movies["adjusted"]
movies_adjusted = pd.DataFrame(movies_adjusted)
movies_adjusted[movies_adjusted["adjusted"]<200000000] = 0
movies_adjusted[movies_adjusted["adjusted"]>200000000] = 1
movies = pd.concat([movies.drop(["adjusted"],axis =1),movies_adjusted], axis =1)
# set adjusted = 200000000 as the threshold, Set the predicted data to 0 and 1 respectively.

In [18]:
y_data = movies["adjusted"]
x_data = movies.drop(["adjusted"], axis=1)
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=42,sampling_strategy='minority', n_neighbors=5, n_jobs=1)
x_data, y_data = ada.fit_resample(x_data, y_data)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_data, y_data, test_size=0.2)
# split the data into train and test data
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

Using TensorFlow backend.


(549, 27)
(549,)
(138, 27)
(138,)


In [19]:
# the first is SVC
# tunning the hyperparameters and use r2 score and mean_squared_error to evaluate the results
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
tuned_parameters =  [{'kernel': ['rbf'], 'gamma': [10, 1, 1e-1, 1e-2, 1e-3, 1e-4],
                     'C': [1, 10, 100, 100]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 100],'gamma': [100, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4]}]

scores = ["accuracy","f1_macro"]

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print("\n")
    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring= score)
    clf.fit(x_train, y_train)
    print("Best parameters set found on the training set:")
    print(clf.best_params_)
    print("\n")

# Tuning hyperparameters for accuracy


Best parameters set found on the training set:
{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


# Tuning hyperparameters for f1_macro


Best parameters set found on the training set:
{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}




In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# use the tuning hyperparameters to build the final model
svm = SVC(C = 1, kernel = "rbf", gamma = 0.1 )
# fit the model using some training data
svm_fit = svm.fit(x_train, y_train)
# generate a mean accuracy score for the predicted data
train_score = svm.score(x_train, y_train)

# print the mean accuracy of testing predictions
print("Accuracy score = " + str(round(train_score, 4)))

# predict the test data
predicted = svm.predict(x_test)

# generate a mean accuracy score for the predicted data
test_score = svm.score(x_test, y_test)

# print the mean accuracy of testing predictions
print("Accuracy score = " + str(round(test_score, 4)))
print(classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))
# scource: moodle warwick

Accuracy score = 0.9381
Accuracy score = 0.8913
              precision    recall  f1-score   support

         0.0       0.94      0.84      0.89        70
         1.0       0.85      0.94      0.90        68

    accuracy                           0.89       138
   macro avg       0.89      0.89      0.89       138
weighted avg       0.90      0.89      0.89       138

[[59 11]
 [ 4 64]]


In [22]:
# the secone model is xgboosting
import xgboost as xgb
param_test1 = {
 'learning_rate':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1],'subsample':[i/10.0 for i in range(6,10)]
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=7,reg_alpha= 0.1,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_train,y_train)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([0.22231178, 0.21722474, 0.19127755, 0.23636203, 0.15822716,
         0.14671378, 0.21446061, 0.19998541, 0.20703673, 0.10757976,
         0.15616736, 0.21011367, 0.21326823, 0.13553987, 0.16087298,
         0.21219711, 0.13716497, 0.16150374, 0.18997931, 0.18799124,
         0.13994212, 0.22746201, 0.13818407, 0.21480341, 0.15768523,
         0.2220983 , 0.19129643, 0.12719707, 0.22036877, 0.24450436,
         0.14530015, 0.25235462, 0.14661026, 0.28045287, 0.26559076,
         0.22335196, 0.18625078, 0.22205825, 0.19191132, 0.16629524]),
  'std_fit_time': array([0.1210614 , 0.10471801, 0.07112361, 0.09996822, 0.04830525,
         0.04183608, 0.1603343 , 0.16067102, 0.19896685, 0.01568447,
         0.04789325, 0.06830734, 0.16137114, 0.03634789, 0.06868215,
         0.14350233, 0.04334482, 0.05661549, 0.10919914, 0.05655372,
         0.02717555, 0.13473994, 0.05292796, 0.09710718, 0.08276278,
         0.18551764, 0.05307404, 0.0161249 , 0.12812415, 0.22711793,

In [23]:
param_test2 = {
 'colsample_bytree':[i/10.0 for i in range(6,10)],'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],'max_depth':list(range(3,10,2))
}
gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.04, n_estimators=140, max_depth=7,reg_alpha= 0.1,
 min_child_weight=1, gamma=0, subsample=0.7, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(x_train,y_train)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

({'mean_fit_time': array([0.150769  , 0.15156908, 0.15098953, 0.15598807, 0.05827122,
         0.20347676, 0.10227776, 0.20931802, 0.12343369, 0.09576588,
         0.25820804, 0.10488214, 0.16686997, 0.18051348, 0.07797079,
         0.27169366, 0.16647263, 0.2104466 , 0.21740918, 0.072925  ,
         0.09516473, 0.11930132, 0.12788939, 0.10516205, 0.07706203,
         0.13961973, 0.13789368, 0.19500728, 0.19925952, 0.07376137,
         0.15063791, 0.25981636, 0.1709497 , 0.15280938, 0.07844424,
         0.24699559, 0.21495113, 0.26780272, 0.25536313, 0.10249462,
         0.14410324, 0.13486881, 0.2207942 , 0.17867975, 0.07668724,
         0.25876169, 0.20421348, 0.23748856, 0.16646624, 0.1343751 ,
         0.2428134 , 0.28647118, 0.35937028, 0.23006639, 0.10039134,
         0.30037985, 0.37414069, 0.27983956, 0.29893332, 0.12095022,
         0.17515793, 0.18947463, 0.14495497, 0.16065288, 0.08752165,
         0.18151355, 0.27760863, 0.29179063, 0.28000097, 0.13293772,
         0.247825

In [28]:
param_test3 = {
  'min_child_weight':list(range(1,6,2)),'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.04, n_estimators=140, max_depth=9,reg_alpha= 0.1,
 min_child_weight=1, gamma=0, subsample=0.7, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(x_train,y_train)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

({'mean_fit_time': array([0.25453157, 0.30398035, 0.11866107, 0.1959353 , 0.17131119,
         0.18117757, 0.25529542, 0.23777308, 0.26270638, 0.30693336,
         0.22443333, 0.20856581, 0.27965012, 0.22988119, 0.14890008]),
  'std_fit_time': array([0.06010042, 0.08548189, 0.04403994, 0.06352136, 0.05350969,
         0.04331446, 0.04792554, 0.0496157 , 0.06049524, 0.09922321,
         0.11202656, 0.06482128, 0.08823344, 0.08215914, 0.06434977]),
  'mean_score_time': array([0.00274563, 0.00256743, 0.00248156, 0.00277047, 0.00254622,
         0.00251565, 0.00256104, 0.00267601, 0.00267682, 0.00283561,
         0.00258718, 0.00258932, 0.00304112, 0.00254164, 0.00236735]),
  'std_score_time': array([1.97273243e-04, 1.15965640e-04, 1.03482848e-04, 2.00820092e-04,
         2.34349314e-04, 8.65330831e-05, 1.50112882e-04, 2.81215538e-04,
         2.26984835e-04, 3.81476480e-04, 1.33106221e-04, 2.27404066e-04,
         4.88776221e-04, 2.29948044e-04, 3.09306170e-04]),
  'param_gamma': masked_a

In [30]:
from sklearn import metrics
def modelfit(alg, x_trian,y_train,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(x_train, y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'],
                          nfold=cv_folds,metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(x_train, y_train, eval_metric='auc')
        
    #Predict training set:
    movies_predictions = alg.predict(x_train)
    movies_predprob = alg.predict_proba(x_train)[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, movies_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, movies_predprob))

In [34]:
import xgboost as xgb
xgb1 = xgb.XGBClassifier(learning_rate =0.04, n_estimators=140, max_depth=9,reg_alpha= 0.1,min_child_weight=1, gamma=0, subsample=0.7, colsample_bytree=0.6,objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)
modelfit(xgb1, x_train,y_train)


Model Report
Accuracy : 0.9836
AUC Score (Train): 0.998553


In [35]:
predictions = xgb1.predict(x_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.86      0.91      0.89        70
         1.0       0.91      0.85      0.88        68

    accuracy                           0.88       138
   macro avg       0.89      0.88      0.88       138
weighted avg       0.89      0.88      0.88       138

[[64  6]
 [10 58]]


In [42]:
# the third model will be GBDT,tunning the hyperparameters.
from sklearn.ensemble import GradientBoostingClassifier
param_test1 = {'learning_rate':[0.1,0.3,0.5,0.7,0.9],'n_estimators':range(10,81,5)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,n_estimators = 50,max_features= 9
                                  min_samples_leaf=20,max_depth=8,max_features=9, subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',iid=False,cv=5)
gsearch1.fit(x_train,y_train)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([0.01040549, 0.0096839 , 0.01042132, 0.01401129, 0.01651096,
         0.0179739 , 0.01944547, 0.02139821, 0.02410226, 0.02687306,
         0.02857566, 0.034097  , 0.03344855, 0.03942523, 0.0382123 ,
         0.00565424, 0.00811067, 0.01052699, 0.01327519, 0.01491742,
         0.01695485, 0.01938934, 0.02465906, 0.02459641, 0.02701001,
         0.02922683, 0.03143373, 0.03395376, 0.03536391, 0.03876824,
         0.00584297, 0.00864844, 0.01061277, 0.01273856, 0.01472387,
         0.01704793, 0.01890554, 0.02160101, 0.02346411, 0.0262846 ,
         0.02821212, 0.03056884, 0.03332934, 0.03529987, 0.03844252,
         0.00572824, 0.00785632, 0.01040573, 0.01301303, 0.0148807 ,
         0.01671925, 0.01865983, 0.0209693 , 0.02413788, 0.02893395,
         0.02822499, 0.03102612, 0.03303332, 0.03566084, 0.03733597,
         0.00544524, 0.00767784, 0.0103034 , 0.01235933, 0.01434956,
         0.01682377, 0.01950479, 0.02132454, 0.02417812, 0.0273458 ,
         0.029211

In [44]:
param_test2 = {'max_depth':range(3,20,2), 'min_samples_split':range(100,1001,200), 'min_samples_leaf':range(50,81,10)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.5, min_samples_split=300,n_estimators = 65,max_features= 9,
                                  min_samples_leaf=20,max_depth=8,max_features=9, subsample=0.8,random_state=10), 
                       param_grid = param_test2, scoring='roc_auc',iid=False,cv=5)
gsearch2.fit(x_train,y_train)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

({'mean_fit_time': array([0.03857379, 0.03192272, 0.02606397, 0.024435  , 0.02385912,
         0.03085642, 0.0278017 , 0.02409902, 0.02416492, 0.02436323,
         0.03001671, 0.02772846, 0.02389188, 0.02389054, 0.02398262,
         0.02919483, 0.02738166, 0.02408729, 0.02422738, 0.02395053,
         0.03177638, 0.0278945 , 0.02400942, 0.02412057, 0.02401896,
         0.0346189 , 0.02966619, 0.02431035, 0.02508621, 0.02551365,
         0.03196282, 0.02946835, 0.02548742, 0.02530966, 0.0253562 ,
         0.03075342, 0.03376055, 0.02594438, 0.02719007, 0.0288424 ,
         0.03677979, 0.03165884, 0.02693696, 0.02477341, 0.02497306,
         0.03236914, 0.02969794, 0.02633891, 0.02549424, 0.02503648,
         0.03062916, 0.02886605, 0.02432432, 0.02559681, 0.02569427,
         0.03179088, 0.03029284, 0.02537985, 0.02700901, 0.02524123,
         0.03494239, 0.02974315, 0.02511153, 0.02781177, 0.02563081,
         0.03294821, 0.03011003, 0.02482195, 0.02540784, 0.02567873,
         0.031079

In [46]:
param_test3 = {'max_features':range(9,15,2),'subsample':[0.1,0.3,0.5,0.7,0.9],'min_samples_split':range(10,101,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.5, min_samples_split=100,n_estimators = 65,
                                  min_samples_leaf=60,max_depth=5,max_features=9, subsample=0.8,random_state=10), 
                       param_grid = param_test3, scoring='roc_auc',iid=False,cv=5)
gsearch3.fit(x_train,y_train)
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

({'mean_fit_time': array([0.02993999, 0.03205519, 0.0324954 , 0.03843536, 0.03776169,
         0.02444453, 0.02951221, 0.03249092, 0.0355649 , 0.03652182,
         0.02350922, 0.02777905, 0.03053422, 0.03388534, 0.03767014,
         0.02464924, 0.0294714 , 0.03195071, 0.03399105, 0.03860445,
         0.02506022, 0.02943478, 0.03234677, 0.03580546, 0.03777051,
         0.02339282, 0.02820759, 0.0307198 , 0.03445749, 0.04213743,
         0.02553148, 0.03374329, 0.03371482, 0.03647699, 0.03833971,
         0.0248518 , 0.02999797, 0.03255553, 0.03660188, 0.03993897,
         0.02442417, 0.03105378, 0.03309302, 0.03777509, 0.03727722,
         0.02865505, 0.03073616, 0.03181806, 0.03579192, 0.03900161,
         0.02528815, 0.02965035, 0.03275924, 0.03578744, 0.039821  ,
         0.02459426, 0.02921886, 0.03201485, 0.03587618, 0.04005299,
         0.02441807, 0.02855792, 0.03247576, 0.03571262, 0.0400538 ,
         0.02501197, 0.02850828, 0.03769207, 0.03987098, 0.04115062,
         0.025810

In [52]:
gbc = GradientBoostingClassifier(learning_rate=0.5,n_estimators = 65,max_features= 9,min_samples_leaf=60,max_depth=5,subsample=0.9,random_state=10)
gbc.fit(x_train,y_train)
movies_pred = gbc.predict(x_train)
movies_predprob = gbc.predict_proba(x_train)[:,1]
print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, movies_pred))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, movies_predprob))

Accuracy : 0.9344
AUC Score (Train): 0.983725


In [57]:
predicitons_gbc = gbc.predict(x_test)
print(classification_report(y_test,predicitons_gbc))
print(confusion_matrix(y_test, predicitons_gbc))

              precision    recall  f1-score   support

         0.0       0.81      0.83      0.82        70
         1.0       0.82      0.79      0.81        68

    accuracy                           0.81       138
   macro avg       0.81      0.81      0.81       138
weighted avg       0.81      0.81      0.81       138

[[58 12]
 [14 54]]


In [58]:
# the last step is voting.
from sklearn.ensemble import VotingClassifier
voting_clf= VotingClassifier(estimators=[('svm', svm),('xgb1', xgb1),('gbc',gbc)])
voting_clf.fit(x_train,y_train)
votingg_pred = voting_clf.predict(x_test)
print(votingg_pred)
print(classification_report(y_test, votingg_pred))
print(confusion_matrix(y_test, votingg_pred))

[1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0.]
              precision    recall  f1-score   support

         0.0       0.88      0.87      0.88        70
         1.0       0.87      0.88      0.88        68

    accuracy                           0.88       138
   macro avg       0.88      0.88      0.88       138
weighted avg       0.88      0.88      0.88       138

[[61  9]
 [ 8 60]]
